In [ ]:
import pandas as pd
from datetime import datetime
import torch.nn.functional as F
import torch
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from scipy.special import logit, expit
from scipy.stats import spearmanr, kendalltau

## Data preparation

In [ ]:
!wget https://www.dropbox.com/s/s4qj0fpsn378m2i/chgk.zip -nc
!unzip chgk.zip -d ./

File ‘chgk.zip’ already there; not retrieving.

Archive:  chgk.zip
replace ./players.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
players_dict = pd.read_pickle("players.pkl")
results_dict = pd.read_pickle("results.pkl")
tournaments_dict = pd.read_pickle("tournaments.pkl")

In [ ]:
players = pd.DataFrame.from_dict(players_dict, orient='index')
tournaments = pd.DataFrame.from_dict(tournaments_dict, orient='index')

In [ ]:
tournaments["year"] = tournaments["dateStart"].apply(lambda x : datetime.fromisoformat(x).year)
tournaments_2019_2020 = tournaments.query("year == 2019 or year == 2020")[['id', 'name', 'year']]
print(tournaments_2019_2020.shape)
tournaments_2019_2020.head()

(1105, 3)


,id,name,year
4628,4628,Семь сорок,2020
4772,4772,Синхрон северных стран. Зимний выпуск,2019
4957,4957,Синхрон Биркиркары,2020
4973,4973,Балтийский Берег. 3 игра,2019
4974,4974,Балтийский Берег. 4 игра,2019


In [ ]:
tournaments_2019_2020_ids = set(tournaments_2019_2020.id.tolist())

In [ ]:
print(players.shape)
players.head()

(204063, 4)


,id,name,patronymic,surname
1,1,Алексей,None,Абабилов
10,10,Игорь,,Абалов
11,11,Наталья,Юрьевна,Абалымова
12,12,Артур,Евгеньевич,Абальян
13,13,Эрик,Евгеньевич,Абальян


Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl).

In [ ]:
def has_mask(item):
  return 'mask' in item.keys() and item['mask'] is not None

def has_members(item):
  return 'teamMembers' in item.keys() and len(item['teamMembers']) > 0

results_dict_filtered = {k: v for k,v in results_dict.items() if (all([has_mask(x) and has_members(x) for x in v]) and k in tournaments_2019_2020_ids)}

In [ ]:
temp = []
for tournament_id, payload in results_dict_filtered.items():
    for team in payload:
      mask = team['mask']
      position = team['position']
      team_id = team['team']['id']
      team_name = team['team']['name']
      for m in team['teamMembers']:
        player_id = m['player']['id']
        line = [tournament_id, mask, position, team_id, team_name, player_id]
        temp += [line]

results = pd.DataFrame(temp, columns=['tournament_id', 'mask', 'position', 'team_id', 'team_name', 'player_id'])

In [ ]:
results = pd.merge(results, tournaments_2019_2020[['id', 'year']], left_on='team_id', right_on='id').drop(['id'], axis=1)
print(results.shape)
results.head()

(11138, 7)


,tournament_id,mask,position,team_id,team_name,player_id,year
0,4772,101111101111111110001101011001111010,5.5,5444,Эйфью,36742,2019
1,4772,101111101111111110001101011001111010,5.5,5444,Эйфью,28939,2019
2,4772,101111101111111110001101011001111010,5.5,5444,Эйфью,54289,2019
3,4772,101111101111111110001101011001111010,5.5,5444,Эйфью,15381,2019
4,4772,101111101111111110001101011001111010,5.5,5444,Эйфью,27375,2019


In [ ]:
def unruffle(df):
  temp = []
  for index, row in df.iterrows():
    mask = row['mask']
    for question_num, result in enumerate(list(mask)):
      if not (result == '0' or result == '1'):
        continue 
      answered = int(result) == 1
      temp += [[row['tournament_id'], question_num, row['team_id'], row['player_id'], row['year'], answered]]
  return pd.DataFrame(temp, columns=['tournament_id', 'question_num', 'team_id', 'player_id', 'year', 'result'])

In [ ]:
answer_results = unruffle(results)

In [ ]:
print(answer_results.shape)
answer_results.head()

(450461, 6)


,tournament_id,question_num,team_id,player_id,year,result
0,4772,0,5444,36742,2019,True
1,4772,1,5444,36742,2019,False
2,4772,2,5444,36742,2019,True
3,4772,3,5444,36742,2019,True
4,4772,4,5444,36742,2019,True


# Part 1

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [ ]:
answer_results['question_id'] = answer_results.apply(lambda x: str(x['tournament_id']) + '_' + str(x['question_num']), axis=1)
data_train = answer_results[answer_results.year==2019].drop(['year', 'tournament_id', 'question_num'], axis=1)
data_test = answer_results[answer_results.year==2020].drop(['year', 'tournament_id', 'question_num'], axis=1)

In [ ]:
data_train.head()

,team_id,player_id,result,question_id
0,5444,36742,True,4772_0
1,5444,36742,False,4772_1
2,5444,36742,True,4772_2
3,5444,36742,True,4772_3
4,5444,36742,True,4772_4


In [ ]:
onehot = OneHotEncoder(handle_unknown='ignore')
X_train = onehot.fit_transform(data_train[['player_id', 'question_id']])
X_test = onehot.transform(data_test[['player_id', 'question_id']])
y_train = data_train['result']
y_test = data_test['result']

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((320273, 18699), (320273,), (130188, 18699), (130188,))

In [ ]:
baseline_model = LogisticRegression(max_iter=500)
baseline_model.fit(X_train, y_train)

LogisticRegression(max_iter=500)

In [ ]:
baseline_model.score(X_train, y_train)

0.8382754712385996

In [ ]:
baseline_model.score(X_test, y_test)

0.6780117983224261

### Rating

In [ ]:
# calculate rating as coeff in linear part of logistic regression corresponding to an id
user_rating = pd.DataFrame({'id': onehot.categories_[0], 'rating': expit(baseline_model.coef_[0][:len(onehot.categories_[0])])})
question_rating = pd.DataFrame({'id': onehot.categories_[1], 'rating': expit(-baseline_model.coef_[0][len(onehot.categories_[0]):])})

In [ ]:
data_test_proba = data_test.copy()
data_test_proba['proba'] = baseline_model.predict_proba(X_test)[:,1] # probability that user has answered the question (True)
print(data_test_proba.shape)
data_test_proba.head()

(130188, 5)


,team_id,player_id,result,question_id,proba
9182,6462,8153,True,4772_0,0.902942
9183,6462,8153,True,4772_1,0.722799
9184,6462,8153,True,4772_2,0.722799
9185,6462,8153,True,4772_3,0.454270
9186,6462,8153,True,4772_4,0.902942


In [ ]:
pd.merge(players, user_rating, left_on='id', right_on='id').sort_values(by=['rating'], ascending=False).head(10)

,id,name,patronymic,surname,rating
717,185500,Ксения,Сергеевна,Долгополова,0.890114
310,32458,Игорь,Вячеславович,Тюнькин,0.866659
314,32901,Наиль,Евгеньевич,Фарукшин,0.861115
65,6482,Ким,Гагикович,Галачян,0.857130
466,70750,Денис,Андреевич,Галиакберов,0.825574
286,29981,Андрей,Владимирович,Солдатов,0.817843
378,40635,Александра,Владимировна,Балабан,0.814493
352,36754,Анастасия,Викторовна,Шутова,0.811671
330,34936,Кирилл,Александрович,Чернышёв,0.809890
256,27240,Анастасия,Дмитриевна,Рубашкина,0.805780


Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:

- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

1) Тренируем логистическую регрессию; получаем вероятности ответа на вопрос для игрока 

2) Вероятность ответа для команды -> хотя бы один игрок ответил

3) Считаем cумму вероятностей правильных ответов в турнире -> рейтинг команды

4) чем выше рейтинг команды, тем выше должно быть место (ниже позиция)

In [ ]:
# team question prob answer -> at least somebody from the team has answered a particular question
# team position  -> -sum of team question prob answer in this torunament

def get_team_tournament_rating(data):
  temp = data.copy()
  result = []
  for key, rows in temp.groupby(['team_id', 'question_id']):
      team_id, question_id = key[0], key[1]
      not_answered = (1-rows['proba']).prod() # nobody has answered the question
      rating = (1 - not_answered) # at least somebody has answered a question
      result += [[int(team_id), rating, question_id]]
  r = pd.DataFrame(result, columns=['team_id', 'rating', 'question_id'])
  r['tournament_id'] = r['question_id'].apply(lambda x : int(x.split('_')[0]))
  t = r.groupby(['team_id', 'tournament_id'])['rating'].sum().reset_index(name='team_rating')
  return t

team_rating = get_team_tournament_rating(data_test_proba)

results_test = results.query('year==2020')[['tournament_id', 'team_id', 'position']].drop_duplicates()
ttemp = pd.merge(results_test, team_rating).sort_values(by=['team_id'])

In [ ]:
ttemp.head()

,tournament_id,team_id,position,team_rating
540,5217,4957,94.0,30.029860
539,5074,4957,189.5,33.380150
541,5385,4957,81.5,29.602124
542,5415,4957,2.5,40.619950
511,5025,5694,71.0,27.914929


In [ ]:
import numpy as np

def evaluate_metrics(df):
  sp_corrs = []
  kend_corrs = []
  for tournament_id in df['tournament_id'].unique():
    temp = df[df['tournament_id'] == tournament_id]
    # add - to team rating, since position is negatively correlated with the rating
    kend_corr = kendalltau(-df['team_rating'], df['position']).correlation
    sp_corr  = spearmanr(-df['team_rating'], df['position']).correlation
    if kend_corr is not None:
      kend_corrs += [kend_corr]
    if sp_corr is not None:
      sp_corrs += [sp_corr]
  return np.array(sp_corrs).mean(), np.array(kend_corrs).mean()

In [ ]:
sp, kend = evaluate_metrics(ttemp)
print('Spearmans correlation coefficient: %.3f' % sp)
print('Kendall correlation coefficient: %.3f' % kend)

Spearmans correlation coefficient: 0.182
Kendall correlation coefficient: 0.121


## Part 2

Предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды; разработайте EM-схему для обучения этой модели, реализуйте её в коде;
обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
разработайте EM-схему для обучения этой модели, реализуйте её в коде;
обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


Скрытая переменная - игрок X ответил на вопрос Y. Zxy.


Видимая переменния - команда T ответила на вопрос Y. Kty.

P(Zxy|Kty) = P(Kty|Zxy)*P(Zxy)/P(Kty) = P(Zxy)/P(Kty)


P(Zxy) - вероятность того, что X ответил на вопрос Y, то есть по сути результат логистической регрессии выше.


E-шаг - вычисление E(Zxy) = P(Zxy|Kty)


M-шаг - вычисление P(Zxy)

In [ ]:
EPS = 1e-5

def e_step(data):
    data_ = data.copy()
    result = []
    for key, rows in data_.groupby(['team_id', 'question_id']):
        team_id, question_id = key[0], key[1]
        not_answered = (1-rows['zxy_proba']).prod() # nobody has answered the question
        rating = (1 - not_answered) # at least somebody has answered a question
        result += [[int(team_id), rating, question_id]]
    r = pd.DataFrame(result, columns=['team_id', 'kxy_proba', 'question_id'])
    data_ = pd.merge(data_, r)

    data_['zxy_kty_proba'] = data_['zxy_proba']/data_['kxy_proba']
    
    data_.loc[data_['kxy_proba_initial'] == 0, 'zxy_kty_proba'] = 0
    data_['zxy_kty_proba'] = np.clip(data_['zxy_kty_proba'], EPS, 1 - EPS)
    return data_.drop(['kxy_proba'], axis=1)


def m_step(data, onehot):
    data_ = data.copy()
    x_train = onehot.transform(data_[['player_id', 'question_id']])
    y_train = logit(data_['zxy_kty_proba'])
    model = LinearRegression()
    model.fit(x_train, y_train)
    data_['zxy_proba'] = expit(model.predict(x_train))

    return data_, model

In [ ]:
N_EPOCHS = 10

data_test_ = data_test.copy()
data_train_ = data_train.copy()
data_train_['kxy_proba_initial'] = data_train_['result'].apply(lambda x : 1 if x else 0).astype(int)
onehot = OneHotEncoder(handle_unknown='ignore')
x_train = onehot.fit_transform(data_train_[['player_id', 'question_id']])
# some initialization
data_train_['zxy_proba'] = 1
 
for epoch in range(N_EPOCHS):
    print(epoch)
    data_train_ = e_step(data_train_)
    data_train_, model = m_step(data_train_, onehot)

    #evaluate
    x_test = onehot.transform(data_test[['player_id', 'question_id']])
    data_test_['proba'] = expit(model.predict(x_test))
    team_rating = get_team_tournament_rating(data_test_)
    results_test = results.query('year==2020')[['tournament_id', 'team_id', 'position']].drop_duplicates()
    ttemp = pd.merge(results_test, team_rating).sort_values(by=['team_id'])
    sp, kend = evaluate_metrics(ttemp)
    print('Spearmans correlation coefficient: %.3f' % sp)
    print('Kendall correlation coefficient: %.3f' % kend)    

0
Spearmans correlation coefficient: 0.112
Kendall correlation coefficient: 0.070
1
Spearmans correlation coefficient: 0.098
Kendall correlation coefficient: 0.066
2
Spearmans correlation coefficient: 0.179
Kendall correlation coefficient: 0.120
3
Spearmans correlation coefficient: 0.269
Kendall correlation coefficient: 0.179
4
Spearmans correlation coefficient: 0.332
Kendall correlation coefficient: 0.223
5
Spearmans correlation coefficient: 0.375
Kendall correlation coefficient: 0.253
6
Spearmans correlation coefficient: 0.404
Kendall correlation coefficient: 0.272
7
Spearmans correlation coefficient: 0.423
Kendall correlation coefficient: 0.285
8
Spearmans correlation coefficient: 0.437
Kendall correlation coefficient: 0.295
9
Spearmans correlation coefficient: 0.447
Kendall correlation coefficient: 0.302


Целевые метрики растут с увеличением итераций!

## Part 3

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).


Cложность турнира - среднее от сложности вопросов. Сложность вопроса будет пропорциональна коэффициенту при регрессии.

In [ ]:
question_rating = pd.DataFrame({'id': onehot.categories_[1], 'complexity': expit(-model.coef_[len(onehot.categories_[0]):])})

In [ ]:
tournament_rating = question_rating.copy()
tournament_rating['tournament_id'] = tournament_rating['id'].apply(lambda x : int(x.split('_')[0]))
tournament_rating = tournament_rating.groupby(['tournament_id'])['complexity'].mean().reset_index(name='complexity')

In [ ]:
tournament_rating = pd.merge(tournament_rating, tournaments_2019_2020, left_on='tournament_id', right_on='id')[['id', 'name', 'complexity']]

In [ ]:
tournament_rating.sort_values(by=['complexity'], ascending=False).head(10)

,id,name,complexity
328,5947,Чемпионат Мира. Этап 3. Группа С,0.975525
195,5684,Синхрон высшей лиги Москвы,0.967728
7,5025,Кубок городов,0.957157
413,6261,Зеркало Окского марафона. Злой,0.956159
330,5950,Чемпионат Мира. Финал. Группа С,0.954132
323,5940,Чемпионат Мира. Этап 1. Группа С,0.953829
325,5943,Чемпионат Мира. Этап 2 Группа С,0.950600
317,5923,"Фрегат ""Паллада"": синхрон ""Борского корабела""",0.948133
18,5098,"Ра-II: синхрон ""Борского корабела""",0.945959
227,5756,Жизнь и время Михаэля К.,0.929246


Чемпионаты мира попали в топ сложных турниров, соответсвует интуиции.

In [ ]:
tournament_rating.sort_values(by=['complexity'], ascending=True).head(10)

,id,name,complexity
200,5698,(а)Синхрон-lite. Лига старта. Эпизод VII,0.000003
396,6186,Битва при Марафоне,0.095249
52,5313,(а)Синхрон-lite. Лига старта. Эпизод VI,0.166647
409,6228,Синхрон Первенства Сибири,0.166660
410,6239,Кубок ИРМ,0.199977
287,5855,Лига вузов. IV тур,0.222051
208,5726,Первый турнир имени Джоуи Триббиани,0.222195
416,6269,Открытый синхронный кубок Беларуси,0.222230
338,5975,Чемпионат Минска. Лига Б. Тур первый,0.249977
307,5899,"Ничто, нигде, никогда",0.249978


Синхроны лайт попали в топ легких турниров, соответствует интуиции.